# 패키지 불러오기

In [1]:
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# 데이터 로드

In [5]:
data = pd.read_csv('./data/train.csv')
data.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [4]:
data.isna().sum()

id          0
document    0
label       0
dtype: int64

In [5]:
data.shape

(5000, 3)

- 단어 임베딩을 통해 벡터화
- 모델 선정

- 어떤 임베딩 모델을 쓸 것인지
- 어떤 모델을 쓸 것인지 -> 자연어 처리에 좋은 모델 구상

# 데이터 나누기

## train, test 데이터 나누기

In [7]:
train, val = train_test_split(data, test_size=0.33, random_state=42)

In [8]:
train.shape

(3350, 3)

In [9]:
val.shape

(1650, 3)

# 데이터 정제

## 인덱스 초기화

In [10]:
train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

## 한글 & 공백 제거

In [10]:
train['preprocessed'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
train['preprocessed2'] = train['preprocessed'].str.replace(" +", " ") # 다중 공백 제거
train.head() # 데이터 확인

,id,document,label,preprocessed,preprocessed2
1522,1523,극장에서 보신분들께 심심한 위로의 말을 전합니다.,0,극장에서 보신분들께 심심한 위로의 말을 전합니다,극장에서 보신분들께 심심한 위로의 말을 전합니다
835,836,킬링타임용 그 이상 그 이하도 아님,0,킬링타임용 그 이상 그 이하도 아님,킬링타임용 그 이상 그 이하도 아님
358,359,디워 만도 못한영화는 오랜 만이다,0,디워 만도 못한영화는 오랜 만이다,디워 만도 못한영화는 오랜 만이다
138,139,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,0,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴
299,300,무료로 올레서 봣는데ㅡ화가난다....난화가나...,0,무료로 올레서 봣는데ㅡ화가난다난화가나,무료로 올레서 봣는데ㅡ화가난다난화가나


# 토큰화

In [11]:
from konlpy.tag import Okt 

okt = Okt() # 인스턴스 할당

## 형태소 분석 okt.morphs()

In [17]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in train['preprocessed2']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해주었습니다)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
train["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,id,document,label,preprocessed,preprocessed2,tokenized_stem
0,1523,극장에서 보신분들께 심심한 위로의 말을 전합니다.,0,극장에서 보신분들께 심심한 위로의 말을 전합니다,극장에서 보신분들께 심심한 위로의 말을 전합니다,극장 에서 보신 분들 께 심심하다 위로 의 말 을 전 하다
1,836,킬링타임용 그 이상 그 이하도 아님,0,킬링타임용 그 이상 그 이하도 아님,킬링타임용 그 이상 그 이하도 아님,킬링타임 용 그 이상 그 이하 도 아니다
2,359,디워 만도 못한영화는 오랜 만이다,0,디워 만도 못한영화는 오랜 만이다,디워 만도 못한영화는 오랜 만이다,디워 만도 못 한 영화 는 오랜 만이 다
3,139,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,0,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,학생 개 답답하다 스토리 어이없다 마지막 학살 웃음 만 나오다
4,300,무료로 올레서 봣는데ㅡ화가난다....난화가나...,0,무료로 올레서 봣는데ㅡ화가난다난화가나,무료로 올레서 봣는데ㅡ화가난다난화가나,무료 로 올레 서 봣는데 ㅡ 화가 난 다난 화가 나


In [18]:
train['tokenized_stem'].isna().sum()

0

## 형태소 분석 후 원하는 품사만 가져오기  okt.pos()

In [12]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in train['document']: # 리뷰들을 하나씩 가져옵니다
    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니
train["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,id,document,label,main_pos
0,1523,극장에서 보신분들께 심심한 위로의 말을 전합니다.,0,극장 보신 심심한 위로 말 전 합니다
1,836,킬링타임용 그 이상 그 이하도 아님,0,킬링타임 용 그 이상 그 이하 아님
2,359,디워 만도 못한영화는 오랜 만이다,0,디워 만도 못 영화 오랜 만이
3,139,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,0,학생 개 답답함 스토리 어이없음 마지막 학살 웃음 나옴
4,300,무료로 올레서 봣는데ㅡ화가난다....난화가나...,0,무료 올레 봣는데 화가 난다 난 화가


In [13]:
X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.label #training 데이터에서 라벨 추출

In [14]:
# preprocessed
# val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
# val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
# tokenized = []
# for sentence in val['preprocessed']:
#     tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
#     tokenize = " ".join(tokens)
#     tokenized.append(tokenize)
# val["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val.head()

,id,document,label,main_pos
0,1502,영화 잘봤습니다 보면서졸앗네요,0,영화 봤습니다 보면서 졸앗네요
1,2587,대단했지 투유초콜렛 광고도 한번보삼 추억이 새록새록ㅎ,1,대단했지 투유 초콜렛 광고 한번 보삼 추억 새록새록
2,2654,대체 우리보고 어쩌라는거냐........좋은 점수를 줄 수가 없다,0,대체 우리 보고 어쩌 거 좋은 점수 줄 수가 없다
3,1056,지금보기엔별거없음 뻔함. 극적인감동도약하고.,0,지금 보기 별거 없음 뻔함 극적 감동 도약
4,706,십광구네요 정말 굳입니다 추천!,1,광구 정말 굳 입니다 추천


In [15]:
X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.label #validation 데이터에서 라벨 추출

# 워드 임베딩

## CountVectorizer

In [12]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_input = vectorizer.transform(X_train)

In [13]:
X_input2 = vectorizer.transform(X_val)

In [14]:
X_input

<3350x7074 sparse matrix of type '<class 'numpy.int64'>'
	with 19335 stored elements in Compressed Sparse Row format>

## tf_idf

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [77]:
tfidfVec = TfidfVectorizer()
tfidfVec.fit(X_train)
X_input = tfidfVec.transform(X_train)
X_input2 = tfidfVec.transform(X_val)

# 모델 및 학습

In [26]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

## functions

In [36]:
def eval_model(y_val, pred):
    print('accuracy_score: ', accuracy_score(y_val, pred))
    print('precision_score: ', precision_score(y_val, pred))
    print('recall_score: ', recall_score(y_val, pred))
    print('f1_score: ', f1_score(y_val, pred))

## 랜덤포레스트

In [38]:
from sklearn.ensemble import RandomForestClassifier

In [39]:
clf = RandomForestClassifier(max_depth=6, random_state=42)
clf.fit(X_input, y_train)

RandomForestClassifier(max_depth=6, random_state=42)

In [40]:
pred = clf.predict(X_input2)

In [41]:
clf.score(X_input2, y_val)

0.6957575757575758

In [42]:
eval_model(y_val, pred)

accuracy_score:  0.6957575757575758
precision_score:  0.8606356968215159
recall_score:  0.44165621079046424
f1_score:  0.5837479270315091


## xgboost

In [31]:
import xgboost as xgb

## XGBClassifier

In [57]:
xgb_model = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=6)

In [58]:
xgb_model.fit(X_input, y_train)

[15:22:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=6,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [59]:
pred = xgb_model.predict(X_input2)

In [60]:
eval_model(y_val, pred)

accuracy_score:  0.7424242424242424
precision_score:  0.7590529247910863
recall_score:  0.6838143036386449
f1_score:  0.7194719471947195


## LGBM

In [61]:
from lightgbm import LGBMClassifier

In [62]:
lgbm = LGBMClassifier(n_estimators=500)

In [63]:
lgbm.fit(X_input, y_train)
pred = lgbm.predict(X_input2)
pred

array([1, 1, 1, ..., 1, 0, 0], dtype=int64)

In [64]:
eval_model(y_val, pred)

accuracy_score:  0.6787878787878788
precision_score:  0.6920863309352518
recall_score:  0.6035131744040151
f1_score:  0.6447721179624665


## linear logistic

In [75]:
from sklearn.linear_model import LogisticRegression     #모델 불러오기

logistic_model = LogisticRegression()     #객체에 모델 할당
logistic_model.fit(X_input, y_train)  #모델 학습
pred = logistic_model.predict(X_input2)

In [44]:
eval_model(y_val, pred)

accuracy_score:  0.813939393939394
precision_score:  0.8266666666666667
recall_score:  0.7779171894604768
f1_score:  0.8015513897866839


In [38]:
xgb_model2 = xgb.XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=7)

In [39]:
xgb_model2.fit(X_input, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=6,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [40]:
pred = xgb_model2.predict(X_input2)

In [41]:
pred

array([0.5429903 , 0.66942656, 0.42433965, ..., 0.67439574, 0.8019894 ,
       0.21362828], dtype=float32)

In [42]:
pred_df = pd.DataFrame(pred)
pred_df.head()

,0
0,0.542990
1,0.669427
2,0.424340
3,0.517824
4,0.812054


In [43]:
pred_df[1] = round(pred_df[0])

In [52]:
pred_df[1].value_counts()

 0.0    969
 1.0    677
 2.0      3
-1.0      1
Name: 1, dtype: int64

In [45]:
accuracy_score(y_val, pred_df[1])

0.7824242424242425

In [46]:
precision_score(y_val, pred)

0.802731411229135

In [ ]:
recall_score(y_val, pred)

In [48]:
f1_score(y_val, pred)

0.7266483516483516

# 적용

## test 데이터 로드

In [65]:
test = pd.read_csv('./data/test.csv')
test.head()

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다


## 테스트 데이터 품사 태깅

In [66]:
# main_pos
main_pos = []
for sentence in test['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test["main_pos"] = pd.DataFrame(main_pos)

test.head()

,id,document,main_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족 끼리 드라마 보듯이 보면 딱
2,3,Childhood fantasy,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화 입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동 적이네요 펑펑 울었습니다


In [84]:
test.main_pos.isna().sum()

0

In [67]:
X_test = test['main_pos']

## 워드 임베딩

### Vectorizer

In [68]:
X_test_input = vectorizer.transform(X_test)

NameError: name 'vectorizer' is not defined

### tf-idf

In [78]:
# tfidfVec = TfidfVectorizer()
X_test_input = tfidfVec.transform(X_test)
X_test_input

<5000x7074 sparse matrix of type '<class 'numpy.float64'>'
	with 21615 stored elements in Compressed Sparse Row format>

## 예측

In [79]:
pred = logistic_model.predict(X_test_input)

In [80]:
submission = pd.read_csv("./data/sample_submission.csv") #제출용 파일 불러오기
submission.head() #제출 파일이 잘 생성되었는지 확인

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [81]:
submission["label"] = pred #예측 값 넣어주기
submission.head() # 데이터가 잘 들어갔는지 확인합니다.

,id,label
0,1,0
1,2,1
2,3,0
3,4,0
4,5,1


In [82]:
submission.to_csv("./tf_idf_logistic_submission.csv",index=False)